In [1]:
import numpy as np
import matplotlib.pyplot as plt
import jieba
import jieba.analyse
import io
import pickle
import tensorflow as tf

%matplotlib inline

# 读取文件

In [2]:
reload_data = False

if reload_data:
    corpus_raw = open('./honglou80.txt','r').read()

    print("Corpus is {} characters long".format(len(corpus_raw)))

    words = jieba.lcut(corpus_raw,cut_all=False,HMM=True)

    def create_lookup_tables(text):
        """
        Create lookup tables for vocab
        :param text: The GOT text split into words
        :return: A tuple of dicts (vocab_to_int, int_to_vocab)
        """
        vocab = set(text)
        int_to_vocab = {key: word for key, word in enumerate(vocab)}
        vocab_to_int = {word: key for key, word in enumerate(vocab)}
        return vocab_to_int, int_to_vocab

    vocab_to_int, int_to_vocab = create_lookup_tables(words)
    corpus_int = [vocab_to_int[word] for word in words]
    pickle.dump((corpus_int, vocab_to_int, int_to_vocab), open('preprocess.p', 'wb'))
else:
    corpus_int,vocab_to_int,int_to_vocab = pickle.load(open('preprocess.p','rb'))

# Build the Network
### Batch the Data

### Hyperparameters

In [3]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target data
    :param int_text: text with words replaced by their ids
    :param batch_size: the size that each batch of data should be
    :param seq_length: the length of each sequence
    :return: batches of data as a numpy array
    """
    words_per_batch = batch_size * seq_length
    num_batches = len(int_text)//words_per_batch
    int_text = int_text[:num_batches*words_per_batch]
    y = np.array(int_text[1:] + [int_text[0]])
    x = np.array(int_text)
    
    x_batches = np.split(x.reshape(batch_size, -1), num_batches, axis=1)
    y_batches = np.split(y.reshape(batch_size, -1), num_batches, axis=1)
    
    batch_data = list(zip(x_batches, y_batches))
    
    return np.array(batch_data)

In [54]:
num_epochs = 500
batch_size = 128
rnn_size = 1024
num_layers = 3
keep_prob = 0.7
embed_dim = 1024
seq_length = 50
learning_rate = 0.001
save_dir = './save'

### Build the Graph

In [26]:
train_graph = tf.Graph()
with train_graph.as_default():    
    
    # Initialize input placeholders
    input_text = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    
    # Calculate text attributes
    vocab_size = len(int_to_vocab)
    input_text_shape = tf.shape(input_text)
    
    def lstm_cell():
        # Build the RNN cell
        lstm = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_size)
        drop_cell = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop_cell
    
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
    
    # Set the initial state
    initial_state = cell.zero_state(input_text_shape[0], tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    
    # Create word embedding as input to RNN
    embed = tf.contrib.layers.embed_sequence(input_text, vocab_size, embed_dim)
    
    # Build RNN
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    
    # Take RNN output and make logits
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    # Calculate the probability of generating each word
    probs = tf.nn.softmax(logits, name='probs')
    
    # Define loss function
    cost = tf.contrib.seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_text_shape[0], input_text_shape[1]]),
        name='cost'
    )
    
    # Learning rate optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)
    
    # Gradient clipping to avoid exploding gradients
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients,name='train_op')
    
    

### Train the Network

In [ ]:
import time

pickle.dump((seq_length, save_dir), open('params.p', 'wb'))
batches = get_batches(corpus_int, batch_size, seq_length)
num_batches = len(batches)
start_time = time.time()

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})
        
        for batch_index, (x, y) in enumerate(batches):
            feed_dict = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate
            }
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed_dict)
            
        with open('cost.txt','a') as f:
            f.write('%.3f\n'%train_loss)
            
        time_elapsed = time.time() - start_time
        print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}   time_elapsed = {:.3f}   time_remaining = {:.0f}'.format(
            epoch + 1,
            batch_index + 1,
            len(batches),
            train_loss,
            time_elapsed,
            ((num_batches * num_epochs)/((epoch + 1) * (batch_index + 1))) * time_elapsed - time_elapsed))

        # save model every 10 epochs
        if epoch % 10 == 0:
            saver = tf.train.Saver()
            saver.save(sess, save_dir)
            
            print('Model Trained and Saved')
            

Epoch   1 Batch   63/63   train_loss = 7.073   time_elapsed = 43.811   time_remaining = 21861
Model Trained and Saved
Epoch   2 Batch   63/63   train_loss = 6.984   time_elapsed = 88.070   time_remaining = 21929


In [22]:
# reload graph and continue training

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load the saved model
    loader = tf.train.import_meta_graph(save_dir + '.meta')
    loader.restore(sess, save_dir)
    
    # Get tensors from loaded graph
    input_text = loaded_graph.get_tensor_by_name('input:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
    train_op = loaded_graph.get_operation_by_name('train_op:0')
    cost = loaded_graph.get_tensor_by_name('cost:0')
    
    for epoch in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})
        
        for batch_index, (x, y) in enumerate(batches):
            feed_dict = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate
            }
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed_dict)
            
        time_elapsed = time.time() - start_time
        print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}   time_elapsed = {:.3f}   time_remaining = {:.0f}'.format(
            epoch + 1,
            batch_index + 1,
            len(batches),
            train_loss,
            time_elapsed,
            ((num_batches * num_epochs)/((epoch + 1) * (batch_index + 1))) * time_elapsed - time_elapsed))

        # save model every 10 epochs
        if epoch % 10 == 0:
            saver = tf.train.Saver()
            saver.save(sess, save_dir)
            print('Model Trained and Saved')

INFO:tensorflow:Restoring parameters from ./save


TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("targets:0", shape=(?, ?), dtype=int32) is not an element of this graph.

# Generate GOT Text
### Pick a Random Word

In [60]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word with some randomness
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    return np.random.choice(list(int_to_vocab.values()), 1, p=probabilities)[0]


### Load the Graph and Generate

In [86]:
gen_length = 1000
prime_words = '你们东府里除了那两个石头狮子干净，只怕猫儿狗儿都不干净'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load the saved model
    loader = tf.train.import_meta_graph(save_dir + '.meta')
    loader.restore(sess, save_dir)
    
    # Get tensors from loaded graph
    input_text = loaded_graph.get_tensor_by_name('input:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
    
    # Sentences generation setup
    gen_sentences = jieba.lcut(prime_words)
    prev_state = sess.run(initial_state, {input_text: np.array([[1 for word in gen_sentences]])})
    
    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])
        
        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})

        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
        
    # Remove tokens
    chapter_text = ''.join(gen_sentences).replace('||','\r\n')
        
print(chapter_text)

INFO:tensorflow:Restoring parameters from ./save
你们东府里除了那两个石头狮子干净，只怕猫儿狗儿都不干净，我就有你说的；若怕外头如此，我竟不说了。”一面说，一面往外都写。迎春便坐在山石上，说道：“宝二爷去了，我想不到要见你的踢老太太看，也有扇酒，怎么不给我去。”宝玉见说的便送到两个里来，只见兴儿出来时，又有话喝，忽见平儿回说道：“二奶奶打发他到这个来的，自没有借他的话。你是说有人不好。”
周瑞家的道：“姑娘说去。”说着道：“老太太竟了，隔着许多眼，他们又老上说去；他带的小丫头们。我没两个不解，却连他们又丢过奶奶，然后不遂心，小拐你两件若干地方，不拿奶奶手里去。再在外头前，只说是一个：‘累得姑娘们跟了他几个人，他就不肯照管你取笑儿了。”黛玉忙笑道：“既有，当日方才太太见了他姐妹，忙说我们不错，若说‘自古来许多。”宝玉道：“正是的怎么我子呢？”
又道：“小厮们自然这么高兴了，不敢足有一句，这也不是得他的。”凤姐儿道：“我说你再叫他在里头，刚才奶奶已经回了这个：事情怎么了，这样事情，二爷再一书房出来，昨日破了银子给我，也没有‘宝玉’字。”
李纨又笑道：“这是个姑娘跟前才是。——只因你自己一处再去，怎么也不照看得好罢？”宝玉道：“这是什么香？”
一语未了，只见探春道：“有我罢了，咱们我才服他！”宝玉心中急的名，凤姐也来至榻上，去找宝玉，又不好时，不过是催他说呢。此时自是醉了，再只说笑了。
谁知凤姐命他过来，忽见些话的身子都觉，只说无处可，我只同他闹的你们和我琏宝玉的。”贾环听了，便将他手中伏侍“一年、一群人的时候儿，莺儿岂不认得些呢？
这里贾瑞言到宝玉坐了，只得写下。黛玉一面哭，一面又问：“你想什么利害？我才和这里，这是炕上，你倒来搜了？”一面说，一面拉着王善保书房来找王夫人。麝月只问：“你这么谁？怎么就进了茶？就另来了！”宝玉道：“我要说你里头，你说妹妹一句话，我就回来了。”说着，“宝玉又该做什么神呢？”迎春道：“姐姐今儿怎么离了藕香榭？”贾芸笑道：“你瞧他这几日没家给他来？”宝玉因问：“在家里做什么呢？”宝玉在厅上便向里道：“屋里是什么日子？”地下的一个早已不知不看时，独却是《风流硝，内有对人起完，一齐插上完了半夜。
忽见众人在里间屋里房中，只见两三个两处大一个一看，正是：“好一个”，只要手早来才要让。
两个三姐儿唤他的

In [81]:
print("""刚进了门，便放声大哭起来。黛玉正在梳洗才毕，见宝玉这个光景，倒吓了一跳，问：“是怎么了？和谁怄了气了？”
凤姐道：“他拿的我来了。”宝玉道：“你到底都明白，和二姑娘不带来，又不用听见。让他娘的女儿想？因我亲自知道。”尤氏又要家常说，只因王夫人道：“你不要你，又是个多心的一个，连大姨娘不说，做多给我妈，你又不用说说就是了。”贾母听说，只管站起来，只在“府儿了，两边也不知“宝玉实在好的。那小丫头想了，好主意，所以他却不提，忙又问道：“你倒是个事情？”
宝玉细说话，只得忍了一张房中，念道：“大妹妹和岫老舅舅无人，便自己吃着出去传了，这里宝钗往那里去了？别说姑娘不放心，只见凤姐、太太、太太一齐都去。到姐儿床上，三姐儿已过了工夫，我们烦他知道：“你们好生去了，一日回来请安。就是他们，我不跑起老太太和人必不上房。”因纨、迎春、探春、彩云等都捧着瞧，也笑向李纨道：“你奶奶也遇见我们两个去了！那两句话，姑娘都不敢，我也是拾着，你们得罪了这个儿，且也有‘男、里头’，难道就是你的‘贝鸡’，自然你起一半呢！”宝玉听了，忙推众人道：“老太太等我替我躲去了。”说着便走。凤姐自不自在，只得回房找他。兴儿也念在女儿，急的是个玉，见他拿了一爱去，亲自拿毕，忽见宝玉笑道：“我只那里收你的手？说出来，只到给那边去。”王夫人说：“我又不知他是谁。”宝玉忙收了茶，二人说：“你说了，今儿可又不是了。”鸳鸯道：“你难道这个是这个姑娘？”
一时薛姨妈又说道：“你问他做什么？”宝玉又笑道：“你若忘了，我细细的告诉了，又瞧：“好生走。”说着，“嗤”的一声说：“你去瞧我说你看看，我就知道。”一语未了，只见贾政进来，凤姐往下屋里去瞧黛玉。周瑞家的说：“姑娘在这里想，要和你们一处吃好什么没有？”宝玉道：“就说得些不用利害？”凤姐儿笑道：“方才两个症雪，你可知道是我替三一个。”黛玉冷笑道：“你会做过的是什么？”刘老老先叫：“瞎了眼的，我又要这么大，只说这么完了；饶了我罢。”说毕，叫他接过，也想起来回了，因说：“我知道好了，你要‘一处一般，‘姐姐’我就题着，谁不知道呢？我怨不得你要错，只在这里回来呢。”湘云道：“你放心，我这会子再多心，都一日急了我们，前儿告诉我和你孙子好话，别的彼此感动了个梦，细来，就有趣儿。”贾母道：“你又提是了，只是回了你。”要哭道：“这打呢：来的，我只拣你来了。我是笑话不成！”袭人听了此话，问道：“你也是有姑娘的衣裳，说没得话？”宝玉听了，便拉那丫头：“取了来？”宝玉笑道：“你仍旧在家里：我是那里了？”宝玉道：“那里不是了？我不知道别人，须是这么更吵呢！”宝玉道：“撕他就疼我！
——宁可哭了！
他是宝玉，竟是一件我要送来的，老太太自然是和人想呢。”黛玉走的半天，说：“巧也不出门，等主子来！”迎春冷笑道：“什么‘院里’老婆子极妙。”黛玉道：“且再看。”袭人便道：“这是人说的这些：里头是个‘四儿’字，必是一点儿不知道罢了。”宝玉道：“宝‘夫妻’的韵，怎么过呢！你不拣，多谢你做？”
说到这婆子出去了。宝钗手里来了这话，又忙说：“不是什么，何必说‘爱’的事，前儿竟知道了。我刚才还不知道：“是你素日头年纪珠吃了。我又往我""")

刚进了门，便放声大哭起来。黛玉正在梳洗才毕，见宝玉这个光景，倒吓了一跳，问：“是怎么了？和谁怄了气了？”
凤姐道：“他拿的我来了。”宝玉道：“你到底都明白，和二姑娘不带来，又不用听见。让他娘的女儿想？因我亲自知道。”尤氏又要家常说，只因王夫人道：“你不要你，又是个多心的一个，连大姨娘不说，做多给我妈，你又不用说说就是了。”贾母听说，只管站起来，只在“府儿了，两边也不知“宝玉实在好的。那小丫头想了，好主意，所以他却不提，忙又问道：“你倒是个事情？”
宝玉细说话，只得忍了一张房中，念道：“大妹妹和岫老舅舅无人，便自己吃着出去传了，这里宝钗往那里去了？别说姑娘不放心，只见凤姐、太太、太太一齐都去。到姐儿床上，三姐儿已过了工夫，我们烦他知道：“你们好生去了，一日回来请安。就是他们，我不跑起老太太和人必不上房。”因纨、迎春、探春、彩云等都捧着瞧，也笑向李纨道：“你奶奶也遇见我们两个去了！那两句话，姑娘都不敢，我也是拾着，你们得罪了这个儿，且也有‘男、里头’，难道就是你的‘贝鸡’，自然你起一半呢！”宝玉听了，忙推众人道：“老太太等我替我躲去了。”说着便走。凤姐自不自在，只得回房找他。兴儿也念在女儿，急的是个玉，见他拿了一爱去，亲自拿毕，忽见宝玉笑道：“我只那里收你的手？说出来，只到给那边去。”王夫人说：“我又不知他是谁。”宝玉忙收了茶，二人说：“你说了，今儿可又不是了。”鸳鸯道：“你难道这个是这个姑娘？”
一时薛姨妈又说道：“你问他做什么？”宝玉又笑道：“你若忘了，我细细的告诉了，又瞧：“好生走。”说着，“嗤”的一声说：“你去瞧我说你看看，我就知道。”一语未了，只见贾政进来，凤姐往下屋里去瞧黛玉。周瑞家的说：“姑娘在这里想，要和你们一处吃好什么没有？”宝玉道：“就说得些不用利害？”凤姐儿笑道：“方才两个症雪，你可知道是我替三一个。”黛玉冷笑道：“你会做过的是什么？”刘老老先叫：“瞎了眼的，我又要这么大，只说这么完了；饶了我罢。”说毕，叫他接过，也想起来回了，因说：“我知道好了，你要‘一处一般，‘姐姐’我就题着，谁不知道呢？我怨不得你要错，只在这里回来呢。”湘云道：“你放心，我这会子再多心，都一日急了我们，前儿告诉我和你孙子好话，别的彼此感动了个梦，细来，就有趣儿。”贾母道：“你又提是了，只是回了你。”要哭道：“这打呢：来的，我只拣你来了。我是笑话不成！”袭人听了此话，问道：“你

In [69]:
str.replace?

In [58]:
jieba.lcut(prime_words)

['第八十一回']

# Save a Chapter
### Cleanup Data a Bit

In [174]:
chapter_text = ' '.join(gen_sentences)
for key, token in token_dict.items():
    chapter_text = chapter_text.replace(' ' + token.lower(), key)
chapter_text = chapter_text.replace('\n ', '\n')
chapter_text = chapter_text.replace('( ', '(')
chapter_text = chapter_text.replace(' ”', '”')

capitalize_words = ['lannister', 'stark', 'lord', 'ser', 'tyrion', 'jon', 'john snow', 'daenerys', 'targaryen', 'cersei', 'jaime', 'arya', 'sansa', 'bran', 'rikkon', 'joffrey', 
                    'khal', 'drogo', 'gregor', 'clegane', 'kings landing', 'winterfell', 'the mountain', 'the hound', 'ramsay', 'bolton', 'melisandre', 'shae', 'tyrell',
                   'margaery', 'sandor', 'hodor', 'ygritte', 'brienne', 'tarth', 'petyr', 'baelish', 'eddard', 'greyjoy', 'theon', 'gendry', 'baratheon', 'baraTheon',
                   'varys', 'stannis', 'bronn', 'jorah', 'mormont', 'martell', 'oberyn', 'catelyn', 'robb', 'loras', 'missandei', 'tommen', 'robert', 'lady', 'donella', 'redwyne'
                   'myrcella', 'samwell', 'tarly', 'grey worm', 'podrick', 'osha', 'davos', 'seaworth', 'jared', 'jeyne poole', 'rickard', 'yoren', 'meryn', 'trant', 'king', 'queen',
                   'aemon']

for word in capitalize_words:
    chapter_text = chapter_text.replace(word, word.lower().title())

### Save File

In [175]:
import os
version_dir = './generated-book-v1'
if not os.path.exists(version_dir):
    os.makedirs(version_dir)

num_chapters = len([name for name in os.listdir(version_dir) if os.path.isfile(os.path.join(version_dir, name))])
next_chapter = version_dir + '/chapter-' + str(num_chapters + 1) + '.md'
with open(next_chapter, "w") as text_file:
    text_file.write(chapter_text)